In [1]:
from __future__ import annotations
import polars as pl 
import inspect

In [ ]:
test = ["a","b"]

"|".join(test)

In [ ]:
def test() -> list[str]:
    return ["a"]

test.__annotations__.get("return", "") == list[str]

In [ ]:
from typing import Callable, Concatenate

tt:Callable[[], list]
tt = test

In [ ]:
inspect.signature(tt).return_annotation == "list[str]"

In [ ]:
df = pl.DataFrame({
    "a":[1,1,1,1]
    , "b":[2,2,2,2]
    , "c":[3,3,3,3]
})

test = pl.Series("x",["a", "b"])


df.select(["a", "b"])

In [ ]:
df = pl.DataFrame(
    {
        "a": ["a", "b", "a", "b", "c"],
        "b": [1, 2, 1, 3, 3],
        "c": [5, 4, 3, 2, 1],
    }
)

In [ ]:
for f in df.partition_by("a"):
    print(f.shape)

In [ ]:
df.select(
    pl.col(c).n_unique() for c in df.columns
)

In [ ]:
d = pl.DataFrame(
    {
        "id": ["a", "b", "c", "d"],
        "events": [["e1", "e2"], ["e1", "e2"], ["e1", "e3"], [None]],
    }
)

In [ ]:
d.select(
    pl.col("events").list.unique().list.lengths()
)

In [2]:
from sklearn.datasets import make_classification

In [3]:
orig_x, orig_y = make_classification(n_samples = 300_000, n_features = 50, n_informative = 25, n_redundant = 25)
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))

In [5]:
features = df.columns
features.remove("target")

In [49]:
x = df.select(features)
v = pl.Series([2]*len(features))

print(x.shape)
print(v.shape)

(300000, 50)
(50,)


In [50]:
%%timeit
x.select(
    pl.col(c) * v[i] for i, c in enumerate(x.columns)
).fold(lambda s1, s2: s1 + s2)

25 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
%%timeit
x.select(
    sum(pl.col(c) * v[i] for i, c in enumerate(x.columns))
)

13.1 ms ± 719 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
w = pl.Series([2]*200)


""
i64
2
2
2
2
2
2
2
2
2


In [ ]:
def gradient_descent(df:pl.DataFrame, features:list[str], target:str):
    
    x = df.select(features)
    y = df.select(target)

    weights = pl.Series([1]*len(features))
    const = 0.

    
